In [ ]:
from pathlib import Path
import os

In [ ]:
fp = os.path.join(Path().absolute(), "inputs", "input10.txt")
# fp = os.path.join(Path().absolute(), "inputs", "input10_test.txt")
# fp = os.path.join(Path().absolute(), "inputs", "input10_test2.txt")
# fp = os.path.join(Path().absolute(), "inputs", "input10_test3.txt")
# fp = os.path.join(Path().absolute(), "inputs", "input10_test4.txt")

with open(fp, "r") as f:
    data = f.read().split("\n")[:-1]

In [ ]:
data

# Part 1

In [ ]:
symbol_to_dirs = {
    "|": ["n", "s"],
    "-": ["w", "e"],
    "L": ["n", "e"],
    "J": ["n", "w"],
    "7": ["s", "w"],
    "F": ["s", "e"],
    ".": [],
    "S": ["s", "w"] # 7 (actual problem, test 4)
    # "S": ["s", "e"] # test 1, 2
    # "S": ["s", "e"] # test 3
}

In [ ]:
num_rows = len(data)
num_cols = len(data[0])
print(num_rows, num_cols)

In [ ]:
for i, row in enumerate(data):
    if "S" in row:
        j = row.index("S")
        break

pipe = [(i, j)]
start_i = i
start_j = j

if i > 0 and "s" in symbol_to_dirs[data[i - 1][j]]:
    next = (i - 1, j)
    pipe.append(next)
    last_dir = "s"
elif i < num_rows - 1 and "n" in symbol_to_dirs[data[i + 1][j]]:
    next = (i + 1, j)
    pipe.append(next)
    last_dir = "n"
elif j > 0 and "e" in symbol_to_dirs[data[i][j - 1]]:
    next = (i, j - 1)
    pipe.append(next)
    last_dir = "e"
elif j < num_cols - 1 and "w" in symbol_to_dirs[data[i][j + 1]]:
    next = (i, j + 1)
    pipe.append(next)
    last_dir = "w"
else:
    print("no next found")

print(pipe, last_dir)

i, j = next

max_num = 20000
num = 0

while (i != start_i or j != start_j) and num < max_num:
    symbol = data[i][j]

    other_dir = [d for d in symbol_to_dirs[symbol] if d != last_dir][0]
    # print(symbol, other_dir)
    if other_dir == "s":
        i += 1
        last_dir = "n"
    elif other_dir == "n":
        i -= 1
        last_dir = "s"
    elif other_dir == "w":
        j -= 1
        last_dir = "e"
    elif other_dir == "e":
        j += 1
        last_dir = "w"
    else:
        print("unknown other dir")

    # i, j = next
    pipe.append((i, j))
    num += 1
    # print(num, pipe[-1])

    if len(set(pipe)) != len(pipe):
        print("end")
        break

In [ ]:
len(pipe)

In [ ]:
(len(pipe) - 1) / 2

# Part 2

In [ ]:
# number of intersections
(num_rows + 1) * (num_cols + 1)

In [ ]:
def no_blocker(pos1_x, pos1_y, pos2_x, pos2_y, pos1_dir, pos2_dir):
    # check if no valid blocking connection
    pos1 = data[pos1_x][pos1_y]
    pos2 = data[pos2_x][pos2_y]
    return not ((pos1_x, pos1_y) in pipe and (pos2_x, pos2_y) in pipe and pos1_dir in symbol_to_dirs[pos1] and pos2_dir in symbol_to_dirs[pos2])

In [ ]:
outside_intersections_expanded = []
outside_to_expand = [(0, 0)]
max_num_iter = 20_000
num_iter = 0

while len(outside_to_expand) > 0 and num_iter < max_num_iter:
    # print(f"Expanded: {outside_intersections_expanded}, to expand: {outside_to_expand}")
    print(f"Expanded: {len(outside_intersections_expanded)}, to expand: {len(outside_to_expand)}")

    intersection_to_expand = outside_to_expand.pop()
    x, y = intersection_to_expand

    # up
    if x > 0:
        new = (x - 1, y)
        if new not in outside_to_expand and new not in outside_intersections_expanded:
            if y == 0 or y == num_cols:
                # border
                outside_to_expand.append(new)
            else:
                pos1_x, pos1_y = x - 1, y - 1
                pos2_x, pos2_y = x - 1, y
                pos1_dir = "e"
                pos2_dir = "w"
                if no_blocker(pos1_x, pos1_y, pos2_x, pos2_y, pos1_dir, pos2_dir):
                    outside_to_expand.append(new)
    
    # down
    if x < num_rows:
        new = (x + 1, y)
        if new not in outside_to_expand and new not in outside_intersections_expanded:
            if y == 0 or y == num_cols:
                outside_to_expand.append(new)
            else:
                pos1_x, pos1_y = x, y - 1
                pos2_x, pos2_y = x, y
                pos1_dir = "e"
                pos2_dir = "w"

                if no_blocker(pos1_x, pos1_y, pos2_x, pos2_y, pos1_dir, pos2_dir):
                    outside_to_expand.append(new)

    # left
    if y > 0:
        new = (x, y - 1)
        if new not in outside_to_expand and new not in outside_intersections_expanded:
            if x == 0 or x == num_rows:
                outside_to_expand.append(new)
            else:
                pos1_x, pos1_y = x - 1, y - 1
                pos2_x, pos2_y = x, y - 1
                pos1_dir = "s"
                pos2_dir = "n"

                if no_blocker(pos1_x, pos1_y, pos2_x, pos2_y, pos1_dir, pos2_dir):
                    outside_to_expand.append(new)

    # right
    if y < num_cols:
        new = (x, y + 1)
        if new not in outside_to_expand and new not in outside_intersections_expanded:
            if x == 0 or x == num_rows:
                outside_to_expand.append(new)
            else:
                pos1_x, pos1_y = x - 1, y
                pos2_x, pos2_y = x, y
                pos1_dir = "s"
                pos2_dir = "n"

                if no_blocker(pos1_x, pos1_y, pos2_x, pos2_y, pos1_dir, pos2_dir):
                    outside_to_expand.append(new)

    outside_intersections_expanded.append(intersection_to_expand)
    num_iter += 1

print("done", num_iter)

In [ ]:
num_cells_outside = 0
num_cells_inside = 0
for i, row in enumerate(data):
    for j, cell in enumerate(row):
        if (i, j) not in pipe:
            if any(x in outside_intersections_expanded for x in [(i, j), (i + 1, j), (i, j + 1), (i + 1, j + 1)]):
                # cell is outside
                num_cells_outside += 1
            else:
                num_cells_inside += 1

# print(num_cells_outside)
print(num_cells_inside)